# Introduction to Automated Machine Learning (AutoML)

This sample shows how you can use AutoML to automate the process of training custom ML models.

In this case, we want to train an ML model that automatically applies a label to GitHub issues.

## Install NuGet packages

In [20]:
#r "nuget: Microsoft.ML.AutoML, 0.21.0-preview.23266.6"

Installed Packages Microsoft.ML.AutoML, 0.21.0-preview.23266.6

## Add using statements

In [21]:
using System.Threading;
using System.IO;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.AutoML;

## Initialize MLContext

In [22]:
// Initialize MLContext
MLContext ctx = new MLContext();

## Use AutoML to infer column information

In [23]:
// Define data path
var dataPath = Path.GetFullPath(@"../Data/issues_train.tsv");

// Infer column information
ColumnInferenceResults columnInference =
    ctx.Auto().InferColumns(dataPath, separatorChar: '\t', labelColumnName: "Area", groupColumns: false);

In [24]:
columnInference

Microsoft.ML.AutoML.ColumnInferenceResults TextLoaderOptions Microsoft.ML.Data.TextLoader+Options AllowQuoting True AllowSparse False InputSize <null> Separators [ ] DecimalMarker . Columns index value 0 Microsoft.ML.Data.TextLoader+Column DataKind Single Name ID Source index value 0 Microsoft.ML.Data.TextLoader+Range Min 0 Max 0 AutoEnd False VariableEnd False AllOther False ForceVector False KeyCount <null> 1 Microsoft.ML.Data.TextLoader+Column DataKind String Name Area Source index value 0 Microsoft.ML.Data.TextLoader+Range Min 1 Max 1 AutoEnd False VariableEnd False AllOther False ForceVector False KeyCount <null> 2 Microsoft.ML.Data.TextLoader+Column DataKind String Name Title Source index value 0 Microsoft.ML.Data.TextLoader+Range Min 2 Max 2 AutoEnd False VariableEnd False AllOther False ForceVector False KeyCount <null> 3 Microsoft.ML.Data.TextLoader+Column DataKind String Name Description Source index value 0 Microsoft.ML.Data.TextLoader+Range Min 3 Max 3 AutoEnd False VariableEnd False AllOther False ForceVector False KeyCount <null> TrimWhitespace False HasHeader True UseThreads True ReadMultilines False HeaderFile <null> MaxRows <null> EscapeChar " MissingRealsAsNaNs False ColumnInformation Microsoft.ML.AutoML.ColumnInformation LabelColumnName Area UserIdColumnName <null> GroupIdColumnName <null> ItemIdColumnName <null> ExampleWeightColumnName <null> SamplingKeyColumnName <null> CategoricalColumnNames NumericColumnNames [ ID ] TextColumnNames [ Title, Description ] IgnoredColumnNames ImagePathColumnNames

## Load data into IDataView

In [25]:
// Create text loader
TextLoader loader = ctx.Data.CreateTextLoader(columnInference.TextLoaderOptions);

// Load data into IDataView
IDataView data = loader.Load(dataPath);

## Remove columns

In [26]:
var columnsToExclude = new[]{"ID","Description"};

data = ctx.Transforms.DropColumns(columnsToExclude)
    .Fit(data)
    .Transform(data)

Microsoft.ML.Transforms.ColumnSelectingTransformer+SelectColumnsDataTransform CanShuffle False Source Microsoft.ML.Data.TextLoader+BoundLoader CanShuffle False Schema index value 0 ID: Single Name ID Index 0 IsHidden False Type Single RawType System.Single Annotations Schema (empty) 1 Area: String Name Area Index 1 IsHidden False Type String RawType System.ReadOnlyMemory<System.Char> Annotations Schema (empty) 2 Title: String Name Title Index 2 IsHidden False Type String RawType System.ReadOnlyMemory<System.Char> Annotations Schema (empty) 3 Description: String Name Description Index 3 IsHidden False Type String RawType System.ReadOnlyMemory<System.Char> Annotations Schema (empty) InputSchema index value 0 ID: Single Name ID Index 0 IsHidden False Type Single RawType System.Single Annotations Schema (empty) 1 Area: String Name Area Index 1 IsHidden False Type String RawType System.ReadOnlyMemory<System.Char> Annotations Schema (empty) 2 Title: String Name Title Index 2 IsHidden False Type String RawType System.ReadOnlyMemory<System.Char> Annotations Schema (empty) 3 Description: String Name Description Index 3 IsHidden False Type String RawType System.ReadOnlyMemory<System.Char> Annotations Schema (empty) OutputSchema index value 0 Area: String Name Area Index 0 IsHidden False Type String RawType System.ReadOnlyMemory<System.Char> Annotations Schema (empty) 1 Title: String Name Title Index 1 IsHidden False Type String RawType System.ReadOnlyMemory<System.Char> Annotations Schema (empty)

## Split data into train / validation

80% of the dataset is used for training and 20% is used for validation (tuning)

In [27]:
var trainValidationData = ctx.Data.TrainTestSplit(data, testFraction: 0.2);

In [28]:
SweepablePipeline pipeline =
    ctx.Auto().Featurizer(data, columnInformation: columnInference.ColumnInformation)
        .Append(ctx.Transforms.Conversion.MapValueToKey(columnInference.ColumnInformation.LabelColumnName))
        .Append(ctx.Auto().MultiClassification(labelColumnName: columnInference.ColumnInformation.LabelColumnName))
        .Append(ctx.Transforms.Conversion.MapKeyToValue("PredictedLabel"));

## Create AutoML experiment

In [29]:
AutoMLExperiment experiment = ctx.Auto().CreateExperiment();

## Configure experiment settings

In [30]:
experiment
.SetPipeline(pipeline)
.SetMulticlassClassificationMetric(MulticlassClassificationMetric.MacroAccuracy, labelColumn: columnInference.ColumnInformation.LabelColumnName)
.SetTrainingTimeInSeconds(120)
.SetDataset(trainValidationData);

## Configure experiment monitor

In [31]:
public class AutoMLMonitor : IMonitor
{
    private readonly List<TrialResult> _completedTrials;
    private readonly SweepablePipeline _pipeline;

    public AutoMLMonitor(SweepablePipeline pipeline)
    {
        _completedTrials = new List<TrialResult>();
        _pipeline = pipeline;
    }

    public IEnumerable<TrialResult> GetCompletedTrials() => _completedTrials;

    public void ReportBestTrial(TrialResult result)
    {
        return;
    }

    public void ReportCompletedTrial(TrialResult result)
    {
        var trialId = result.TrialSettings.TrialId;
        var timeToTrain = result.DurationInMilliseconds;
        var pipeline = _pipeline.ToString(result.TrialSettings.Parameter);
        Console.WriteLine($"Trial {trialId} finished training in {timeToTrain}ms with pipeline {pipeline}");
        _completedTrials.Add(result);
    }

    public void ReportFailTrial(TrialSettings settings, Exception exception = null)
    {
        if (exception.Message.Contains("Operation was canceled."))
        {
            Console.WriteLine($"{settings.TrialId} cancelled. Time budget exceeded.");
        }
        Console.WriteLine($"{settings.TrialId} failed with exception {exception.Message}");
    }

    public void ReportRunningTrial(TrialSettings setting)
    {
        return;
    }
}

In [32]:
var monitor = new AutoMLMonitor(pipeline);
experiment.SetMonitor(monitor);

## Train the model

In [33]:
var cts = new CancellationTokenSource();
TrialResult experimentResults = await experiment.RunAsync(cts.Token);

Trial 0 finished training in 11009ms with pipeline FeaturizeText=>Concatenate=>Unknown=>FastTreeOva=>Unknown
Trial 1 finished training in 12438ms with pipeline FeaturizeText=>Concatenate=>Unknown=>FastForestOva=>Unknown
Trial 2 finished training in 723ms with pipeline FeaturizeText=>Concatenate=>Unknown=>SdcaMaximumEntropyMulti=>Unknown
Trial 3 finished training in 12862ms with pipeline FeaturizeText=>Concatenate=>Unknown=>FastForestOva=>Unknown
Trial 4 finished training in 5418ms with pipeline FeaturizeText=>Concatenate=>Unknown=>LbfgsLogisticRegressionOva=>Unknown
Trial 5 finished training in 1120ms with pipeline FeaturizeText=>Concatenate=>Unknown=>LightGbmMulti=>Unknown
Trial 6 finished training in 8544ms with pipeline FeaturizeText=>Concatenate=>Unknown=>FastTreeOva=>Unknown
Trial 7 finished training in 9082ms with pipeline FeaturizeText=>Concatenate=>Unknown=>LbfgsMaximumEntropyMulti=>Unknown
Trial 8 finished training in 8459ms with pipeline FeaturizeText=>Concatenate=>Unknown=>S

## Get the best model

In [34]:
var bestModel = experimentResults.Model;
bestModel

index type value 0 Microsoft.ML.Transforms.Text.TextFeaturizingEstimator+Transformer Microsoft.ML.Transforms.Text.TextFeaturizingEstimator+Transformer 1 Microsoft.ML.Data.ColumnConcatenatingTransformer Microsoft.ML.Data.ColumnConcatenatingTransformer 2 Microsoft.ML.Transforms.ValueToKeyMappingTransformer Microsoft.ML.Transforms.ValueToKeyMappingTransformer 3 Microsoft.ML.Data.MulticlassPredictionTransformer<Microsoft.ML.Trainers.OneVersusAllModelParameters> Microsoft.ML.Data.MulticlassPredictionTransformer`1[Microsoft.ML.Trainers.OneVersusAllModelParameters] FeatureColumnName Features FeatureColumnType Vector Dimensions [ 67785 ] IsKnownSize True ItemType Single RawType System.Single Size 67785 RawType Microsoft.ML.Data.VBuffer<System.Single> Model Microsoft.ML.Trainers.OneVersusAllModelParameters 4 Microsoft.ML.Transforms.KeyToValueMappingTransformer Microsoft.ML.Transforms.KeyToValueMappingTransformer

## Display the metric for the best model

In [35]:
experimentResults.Metric

0.6928909901540781

## Try out the model

### Make predictions

In [36]:
var predictions = bestModel.Transform(trainValidationData.TestSet);

### Display prediction results

In [37]:
predictions.Preview().RowView

index value 0 8 columns Values index value 0 [Area, area-Infrastructure] Key Area Value area-Infrastructure 1 [Area, 4] Key Area Value 4 2 [Title, Add Linux/Mac build script] Key Title Value Add Linux/Mac build script 3 [Title, Sparse vector of size 67785, 30 explicit values] Key Title Value Sparse vector of size 67785, 30 explicit values IsDense False Length 67785 4 [Features, Sparse vector of size 67785, 30 explicit values] Key Features Value Sparse vector of size 67785, 30 explicit values IsDense False Length 67785 5 [PredictedLabel, 4] Key PredictedLabel Value 4 6 [PredictedLabel, area-Infrastructure] Key PredictedLabel Value area-Infrastructure 7 [Score, Dense vector of size 22] Key Score Value Dense vector of size 22 IsDense True Length 22 1 8 columns Values index value 0 [Area, area-System.Numerics] Key Area Value area-System.Numerics 1 [Area, 2] Key Area Value 2 2 [Title, Made Quarternion's operator overloads use their respective methods] Key Title Value Made Quarternion's operator overloads use their respective methods 3 [Title, Sparse vector of size 67785, 75 explicit values] Key Title Value Sparse vector of size 67785, 75 explicit values IsDense False Length 67785 4 [Features, Sparse vector of size 67785, 75 explicit values] Key Features Value Sparse vector of size 67785, 75 explicit values IsDense False Length 67785 5 [PredictedLabel, 17] Key PredictedLabel Value 17 6 [PredictedLabel, area-System.Net] Key PredictedLabel Value area-System.Net 7 [Score, Dense vector of size 22] Key Score Value Dense vector of size 22 IsDense True Length 22 2 8 columns Values index value 0 [Area, area-System.Xml] Key Area Value area-System.Xml 1 [Area, 1] Key Area Value 1 2 [Title, Add async document/element loading for XLinq.] Key Title Value Add async document/element loading for XLinq. 3 [Title, Sparse vector of size 67785, 48 explicit values] Key Title Value Sparse vector of size 67785, 48 explicit values IsDense False Length 67785 4 [Features, Sparse vector of size 67785, 48 explicit values] Key Features Value Sparse vector of size 67785, 48 explicit values IsDense False Length 67785 5 [PredictedLabel, 5] Key PredictedLabel Value 5 6 [PredictedLabel, area-System.Linq] Key PredictedLabel Value area-System.Linq 7 [Score, Dense vector of size 22] Key Score Value Dense vector of size 22 IsDense True Length 22 3 8 columns Values index value 0 [Area, area-System.Xml] Key Area Value area-System.Xml 1 [Area, 1] Key Area Value 1 2 [Title, Wrong named tests in System.Xml.XmlDocument] Key Title Value Wrong named tests in System.Xml.XmlDocument 3 [Title, Sparse vector of size 67785, 47 explicit values] Key Title Value Sparse vector of size 67785, 47 explicit values IsDense False Length 67785 4 [Features, Sparse vector of size 67785, 47 explicit values] Key Features Value Sparse vector of size 67785, 47 explicit values IsDense False Length 67785 5 [PredictedLabel, 1] Key PredictedLabel Value 1 6 [PredictedLabel, area-System.Xml] Key PredictedLabel Value area-System.Xml 7 [Score, Dense vector of size 22] Key Score Value Dense vector of size 22 IsDense True Length 22 4 8 columns Values index value 0 [Area, area-System.Runtime] Key Area Value area-System.Runtime 1 [Area, 14] Key Area Value 14 2 [Title, Consider adding API to return common Funcs and Actions without reallocating] Key Title Value Consider adding API to return common Funcs and Actions without reallocating 3 [Title, Sparse vector of size 67785, 83 explicit values] Key Title Value Sparse vector of size 67785, 83 explicit values IsDense False Length 67785 4 [Features, Sparse vector of size 67785, 83 explicit values] Key Features Value Sparse vector of size 67785, 83 explicit values IsDense False Length 67785 5 [PredictedLabel, 17] Key PredictedLabel Value 17 6 [PredictedLabel, area-System.Net] Key PredictedLabel Value area-System.Net 7 [Score, Dense vector of size 22] Key Score Value Dense vector of size 22 IsDense True Length 22 5 8 columns Values index value 0 [Area, area-System.Text

## Save the best model

In [38]:
ctx.Model.Save(bestModel, data.Schema, "model.mlnet");